# Ingesting

In [58]:
# Imports
import sqlite3
import pandas as pd
from pathlib import Path

# Constants
DATA_FILE_PATH = Path.cwd().parent.joinpath("data").joinpath("raw").joinpath("2024_case_cientista_de_dados_ia.csv")
DATABASES_PATH = Path.cwd().parent.joinpath("data").joinpath("databases")

# Data schema
data_file_data_types = {
    "consumidor_id": "int64",
    "cpf_cnpj": "string",
    "nome do consumidor": "string",
    "perfil": "string",
    "divida_id": "string",
    "código_contrato": "string",
    "valor_vencido": "float64",
    "valor_multa": "float64",
    "valor_juros": "float64",
    "produto": "string",
    "loja": "string",
    "Opcoes_Pagamento": "string",
}

# Ingesting datas do DataFrame
df = pd.read_csv(
    filepath_or_buffer=DATA_FILE_PATH,
    dtype=data_file_data_types,
    parse_dates=["data_nascimento", "data_origem"],
    dayfirst=True,
    decimal=",",
)

# Connection to database (SQLite)
info_db_connection = sqlite3.connect(DATABASES_PATH.joinpath("info.db"), check_same_thread=False)

# Writing table to database
df.to_sql(name="customers", con=info_db_connection, if_exists="replace", index=False)

11

# Prompts

In [49]:
# Basic Imports
from textwrap import dedent

authenticator_prompt_content = dedent("""\
            Definição: 
            Você é um atendente de usuários que são consumidores procurando informações sobre suas dívidas.
                                            
            Objetivo: obter o CPF e data de nascimento dos usuários para autenticá-los. Para isso, interaja com o usuário.
                                    
            Instruções: 
            - O CPF PODE conter entre 10 e 11 dígitos númericos. O usuário pode enviar esse CPF somente em números (exemplo: 74132341062 ou 7413234106) ou com pontuação (exemplo: 338.013.350-70). 
            - A data de nascimento também pode ser enviada em diferentes formatos (exemplos: 14/05/2001, 25-02-2000, 14/05/97, 1989-06-06). 
            - Independente do formato enviado pelo usuário, converta a data para YYYY-MM-DD. Você não precisa informar ao usuário sobre essa conversão.
            - Considere que os usuários são brasileiros, então é comum que o dia seja apresentado antes do mês na data de nascimento.
            - Se não estiver confiante que o usuário lhe forneceu um CPF e data de nascimento seguindo as regras acima, peça esses dados novamente para ele.
            - Quando obter o CPF e data de nascimento do usuário, você deve chamar a função 'autenticar_usuario'.
            
            Extras:
            - Você sempre deve responder no idioma Português Brasileiro.
            - Hoje é {today}."""
        )

information_prompt_content = dedent("""\
            Definição: 
            Você é um atendente de usuários que são consumidores procurando informações sobre suas dívidas.
                                            
            Objetivo: 
            Fornecer informações sobre a dívida o usuário. Utilize o contexto do array de JSONs abaixo para extrair todas as informações das dívidas e possíveis opções de pagamento:
                                            
            {payment_options}

            Dados presentes no array de JSONs:
            - opcao_pagamento_id: não informe esse dado ao usuário.
            - valor_entrada: valor que o usuário terá que pagar de entrada.
            - valor_parcela: valor de cada parcela.
            - valor_desconto: valor do desconto total sobre o campo 'valor_negociado'.
            - valor_negociado: valor total da dívida.
            - quantidade_parcelas: número de parcelas a serem pagas para quitar a dívida.
            - data_primeiro_boleto: data do primeiro boleto.
                                    
            Instruções: 
            - O nome completo do usuário é {name}. Sempre que adequado, chame-o pelo primeiro nome.
            - A dívida foi iniciada na data de {debt_origin_date} e refere-se ao produto {product} da loja {store}
            - Use somente os dados do array de JSONs de contexto para como fonte dos valores e condições de pagamento da dívida.

            Entre as opções que você pode fornecer, estão:
            - Número de opções de pagamento.
            - Características de cada opção de pagamento.
            - Qual é a opção com maior desconto sobre o valor negociado (apresente a porcentagem total de desconto).
            - Qual é a opção que fornece prazo mais longo para pagamento.
            
            Extras:
            - Você sempre deve responder no idioma Português Brasileiro.
            - Hoje é {today}."""
        )

# Tools

In [50]:
# Basic Imports
import sqlite3
from typing import Type, Optional
from textwrap import dedent
import pandas as pd

# LangChain Imports
from langchain.pydantic_v1 import BaseModel, Field
from langchain.callbacks.manager import CallbackManagerForToolRun
from langchain.tools import BaseTool

# Local Imports
# from ingest import info_db_connection

class BaseSqlLiteTool(BaseModel):
    """Base tool for interacting with SQLite Database."""
    connection: sqlite3.Connection = Field(exclude=True)
    # Pass sqlite3.Connection validation
    class Config(BaseTool.Config):
        pass


class AuthenticateUserInput(BaseModel):
    cpf: str = Field(
        description="string contendo entre 10 e 11 caracteres numéricos referentes ao CPF do usuário."
    )
    data_nascimento: str = Field(
        description="string com a data de nascimento do usuário no formato 'YYYY-MM-DD'"
    )

class AuthenticateUser(BaseSqlLiteTool, BaseTool):
    name: str = "autenticar_usuario"
    description: str = dedent("""\
        Autentica o usuário. 
        Os inputs são o CPF e a data de nascimento do usuário. 
        Output é uma mensagem informando se o CPF não foi encontrado nos cadastros ou se a data de nascimento não está correta para aquele usuário.
        Ou, caso tenha sido encontrado, o output é um pandas.DataFrame
        """
    )
    args_schema: Type[BaseModel] = AuthenticateUserInput
    def _run(
        self, 
        cpf: str,
        data_nascimento: str,
        run_manager: Optional[CallbackManagerForToolRun]=None,
    ) -> bool:
        cursor = self.connection.cursor()
        if not cursor.execute(f"SELECT * FROM customers WHERE cpf_cnpj = '{cpf}'").fetchone():
            cursor.close()
            return "CPF não encontrado no cadastro da empresa. Checar CPF novamente."
        elif not cursor.execute(f"SELECT * FROM customers WHERE cpf_cnpj = '{cpf}' AND DATE(data_nascimento) = '{data_nascimento}'").fetchone():
            cursor.close()
            return "Os dados informados não foram encontrados no sistema. Checar data de nascimento novamente."
        else:
            df = pd.read_sql_query(f"SELECT * FROM customers WHERE cpf_cnpj = '{cpf}' AND DATE(data_nascimento) = '{data_nascimento}'", self.connection)
            cursor.close()
            return df

# Creating tools instances
authenticate_user_tool = AuthenticateUser(connection=info_db_connection)

# Saving lists of all tools by var and name
tools = [authenticate_user_tool]
tools_by_name = {tool.name: tool for tool in tools}

# Nodes

In [51]:
# Basic Imports
from datetime import datetime

# LangChain Imports
from langchain_core.messages import ToolMessage
from langchain_core.prompts import ChatPromptTemplate

# Local Imports
# from tools import authenticate_user_tool, tools_by_name

# Constants
TODAY = str(datetime.now().date())

# Classes (Nodes)
class StartNode():
    """Starting node. Only required for routing to the correct agent"""
    def __init__(self):
        pass
    def __call__(self, state):
        if not state["is_authenticated"]:
            return {"is_authenticated": False}
        else:
            return state
    

class AuthenticatorAgentNode():
    """Agent responsible for authenticating the user"""
    def __init__(self, model, prompt):
        self.model = model
        self.prompt = prompt

    def __call__(self, state):
        
        authenticator_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.prompt.format(today=TODAY)),
                ("placeholder", "{messages}"),
            ]
        )
        return {"messages": [(authenticator_prompt | self.model.bind_tools([authenticate_user_tool])).invoke(state)]}
    
    
class AuthenticatorToolNode():
    """Tool execution node for AuthenticatorAgentNode"""
    def __init__(self):
        pass

    def __call__(self, state):
        tool_call = state["messages"][-1].tool_calls[0]
        tool = tools_by_name[tool_call["name"]]
        args = tool_call["args"]
        id = tool_call["id"]
        observation = tool.invoke(args)
        if isinstance(observation, str):
            state["is_authenticated"] = False
            return {"messages": [ToolMessage(content=observation, tool_call_id=id)]}
        else:
            df = observation
            state["is_authenticated"] = True
            state["cpf"] = df["cpf_cnpj"].item()
            state["data_nascimento"] = df["data_nascimento"].item()
            state["nome"] = df["nome"].item()
            state["opcoes_pagamento"] = df["Opcoes_Pagamento"].str.replace('"',"").str.replace("{","{{").str.replace("}","}}").item()
            state["data_origem_divida"] = df["data_origem"].item()
            state["loja"] = df["loja"].item()
            state["produto"] = df["produto"].item()
            state["messages"] = [ToolMessage(content="Usuário autenticado", tool_call_id=id)]
            return state
        

class AuthenticatorOrInfoRouter():
    """Decision node. Routes to the correct agent according to the state of the graph"""
    def __init__(self):
        pass
    def __call__(self, state):
        if state["is_authenticated"]:
            return "to_info"
        else:
            return "to_auth"
            

class ToolRouter():
    """Tool router for any agent that needs to execute tools"""
    def __init__(self):
        pass
    def __call__(self, state):
        print(state)
        if state["messages"][-1].tool_calls:
            return "to_tool"
        else:
            return "to_user"
        

class InformationAgentNode():
    """Agent responsible for providing information to the user"""
    def __init__(self, model, prompt):
        self.model = model
        self.prompt = prompt

    def __call__(self, state):
        information_prompt = ChatPromptTemplate.from_messages([
            ("system", self.prompt.format(
                payment_options=state["opcoes_pagamento"], 
                product=state["produto"],
                debt_origin_date=state["data_origem_divida"],
                name=state["nome"], 
                store=state["loja"],
                today=TODAY,
            )),
            ("placeholder", "{messages}"),
        ])
        return {"messages": [(information_prompt | self.model).invoke(state)]}

# Graph

In [52]:
# Basic Imports
from typing import Annotated, Union
from typing_extensions import TypedDict
from pathlib import Path
import dotenv

# LangGraph Imports
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver

# LangChain Imports
from langchain_openai import ChatOpenAI

# Local Imports
# from prompts import authenticator_prompt_content, information_prompt_content
# from nodes import (
#     StartNode,
#     AuthenticatorAgentNode,
#     AuthenticatorToolNode,
#     InformationAgentNode,
#     AuthenticatorOrInfoRouter,
#     ToolRouter
# )

# Constants
DATABASES_PATH = Path.cwd().parent.joinpath("data").joinpath("databases")

# Loading env-variables
_ = dotenv.load_dotenv(dotenv.find_dotenv())

# Variables
chatgpt_model = ChatOpenAI(model="gpt-4o", temperature=0, max_retries=2,)
memory = SqliteSaver.from_conn_string(DATABASES_PATH.joinpath("memory.db"))

# Graph State
class State(TypedDict):
    messages: Annotated[list, add_messages]
    is_authenticated: Union[bool, None]
    cpf: Union[str, None]
    data_nascimento: Union[str, None]
    nome: Union[str, None]
    opcoes_pagamento: Union[str, None]
    data_origem_divida: Union[str, None]
    loja: Union[str, None]
    produto: Union[str, None]


# Graph Design
graph_builder = StateGraph(State)

graph_builder.add_node("starting_node",  StartNode())
graph_builder.add_node("authenticator_agent", AuthenticatorAgentNode(model=chatgpt_model, prompt=authenticator_prompt_content))
graph_builder.add_node("authenticator_tool", AuthenticatorToolNode())
graph_builder.add_node("information_agent", InformationAgentNode(model=chatgpt_model, prompt=information_prompt_content))

graph_builder.set_entry_point("starting_node")
graph_builder.add_conditional_edges(
    "starting_node",
    AuthenticatorOrInfoRouter(),
    {"to_auth": "authenticator_agent", "to_info": "information_agent",},
)
graph_builder.add_conditional_edges(
    "authenticator_agent",
    ToolRouter(),
    {"to_tool": "authenticator_tool", "to_user": END,},
)
graph_builder.add_conditional_edges(
    "authenticator_tool",
    AuthenticatorOrInfoRouter(),
    {"to_auth": "authenticator_agent", "to_info": "information_agent",},
)
graph_builder.set_finish_point("information_agent")

# Compiling graph
graph = graph_builder.compile(checkpointer=memory)

In [53]:
import uuid
thread_id = str(uuid.uuid4())

In [54]:
from langchain_core.messages import AIMessage

user_input = "cpf 7871803000 e data de nascimento 06/05/1960"
config = {"configurable": {"thread_id": thread_id}}    

def chat(thread_id, user_input):
    config = {"configurable": {"thread_id": thread_id}}    
    for event in graph.stream({"messages": [("user", user_input)]}, config,):
        if (
            "messages" in list(event.values())[0]
            and isinstance(list(event.values())[0]["messages"][-1], AIMessage)
            and list(event.values())[0]["messages"][-1].content
        ):
            yield list(event.values())[0]["messages"][-1].content

lis1 = []
for event in chat(thread_id, user_input):
    print(event)
    lis1.append(event)

{'messages': [HumanMessage(content='cpf 7871803000 e data de nascimento 06/05/1960', id='2bbd2204-1706-4204-8dfb-74face37a9a7'), AIMessage(content='Para confirmar, o CPF fornecido é 7871803000 e a data de nascimento é 06/05/1960. Vou autenticar essas informações.\n\nUm momento, por favor.', additional_kwargs={'tool_calls': [{'id': 'call_17rEMJ6l8qEWAkNPaUw1oSpE', 'function': {'arguments': '{"cpf":"7871803000","data_nascimento":"1960-05-06"}', 'name': 'autenticar_usuario'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 451, 'total_tokens': 521}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3e7d703517', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97757196-4ea9-4b97-a6ce-dea1f8bbb607-0', tool_calls=[{'name': 'autenticar_usuario', 'args': {'cpf': '7871803000', 'data_nascimento': '1960-05-06'}, 'id': 'call_17rEMJ6l8qEWAkNPaUw1oSpE'}], usage_metadata={'input_tokens': 451, 'output_tokens': 70, 'total_tokens': 521})], 'is

In [57]:
# # Utils
graph.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content='cpf 7871803000 e data de nascimento 06/05/1960', id='2bbd2204-1706-4204-8dfb-74face37a9a7'), AIMessage(content='Para confirmar, o CPF fornecido é 7871803000 e a data de nascimento é 06/05/1960. Vou autenticar essas informações.\n\nUm momento, por favor.', additional_kwargs={'tool_calls': [{'id': 'call_17rEMJ6l8qEWAkNPaUw1oSpE', 'function': {'arguments': '{"cpf":"7871803000","data_nascimento":"1960-05-06"}', 'name': 'autenticar_usuario'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 451, 'total_tokens': 521}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_3e7d703517', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-97757196-4ea9-4b97-a6ce-dea1f8bbb607-0', tool_calls=[{'name': 'autenticar_usuario', 'args': {'cpf': '7871803000', 'data_nascimento': '1960-05-06'}, 'id': 'call_17rEMJ6l8qEWAkNPaUw1oSpE'}], usage_metadata={'input_tokens': 451, 'output_tokens': 70, 'total

In [59]:
RESOURCES_DIR = Path.cwd().parent.joinpath("resources").joinpath("logo_1.png")

In [61]:
str(RESOURCES_DIR)

'/Users/ewerthon/Documents/Materiais/[Recursos] Ciência de Dados/ai-assistant-debt/resources/logo_1.png'